In [ ]:
# Define study area (same as other notebooks)
# Coordinates in WGS1984
point1 = [29.575, 4.766667]  # 29°34'30.00"E, 04°46'0.00"N
point2 = [29.279167, 4.766667]  # 29°16'45.00"E, 04°46'0.00"N
point3 = [29.279167, 5.091667]  # 29°16'45.00"E, 05°05'30.00"N
point4 = [29.575, 5.091667]  # 29°34'30.00"E, 05°05'30.00"N

# Create polygon
study_area = ee.Geometry.Polygon([[
    point1, point2, point3, point4, point1
]])

# Apply 20m buffer
study_area_buffered = study_area.buffer(20)

print("Study area created with 20m buffer")
print(f"Area: {study_area_buffered.area().getInfo() / 1e6:.2f} km²")

# Create map centered on study area
Map = geemap.Map(center=[4.929, 29.427], zoom=13)
Map.addLayer(study_area_buffered, {'color': 'blue'}, 'Study Area (20m buffer)')
Map


## Load Outputs from Previous Analyses

This notebook can load data in two ways:

**Option 1: Load from Earth Engine Assets** (if exported to GEE)
- Upload exported GeoTIFFs to Earth Engine as assets
- Load using `ee.Image.loadGeoTIFF()` or `ee.ImageCollection()`

**Option 2: Load from Exported GeoTIFFs** (if saved to Google Drive)
- Download GeoTIFFs from Google Drive
- Upload to Earth Engine as assets
- Or use local files with `rasterio` and convert to EE Images

**Option 3: Re-run Analysis** (if outputs not exported)
- Re-run key cells from both notebooks
- Use variables directly in this notebook

For this example, we'll use **Option 3** - re-running the key analysis steps to generate the required outputs.


## Create Final Mineralization Maps

This section creates **final mineralization maps** by combining:
- Alteration-based potential (from ASTER)
- Structural controls (lineaments, density, structural features from PALSAR/SRTM)

Following standard practice in mineral exploration literature, mineralization maps incorporate both alteration and structural controls.


In [ ]:
# CREATE FINAL MINERALIZATION MAPS
# Combines alteration + structural controls (standard practice in literature)

print("=" * 60)
print("CREATING FINAL MINERALIZATION MAPS")
print("=" * 60)
print("Integrating: Alteration (ASTER) + Structural Controls (PALSAR/SRTM)")
print("Following standard practice in mineral exploration literature")
print("=" * 60)

def create_final_mineralization_map(alteration_potential, structural_features, lineament_density, study_area):
    """
    Create final mineralization map incorporating alteration + structural controls
    
    Standard Practice (Literature):
    - Alteration mapping = Spectral signatures (from ASTER)
    - Structural controls = Lineaments, faults, fractures (from PALSAR/SRTM)
    - Final mineralization = Weighted combination of both
    
    This follows the approach used in:
    - Gabal Shilman, Egypt studies
    - Abu Marawat, Egypt studies
    - Standard GIS weighted overlay modeling
    """
    
    # Normalize inputs to 0-1 scale
    alteration_norm = alteration_potential.unitScale(0, 1)  # Already 0-1
    
    # Normalize lineament density (0-10/km² to 0-1)
    density_norm = lineament_density.unitScale(0, 10)
    
    # Create structural control score
    # High weight on faults and structural complexity
    structural_score = (
        structural_features.select('Faults').multiply(0.4)  # 40% weight on faults
        .add(structural_features.select('Structural_Complexity').multiply(0.3))  # 30% complexity
        .add(structural_features.select('Shear_Zones').multiply(0.2))  # 20% shear zones
        .add(density_norm.multiply(0.1))  # 10% general density
    ).unitScale(0, 1).rename('Structural_Control_Score')
    
    # Final mineralization score: Weighted combination
    # Literature suggests 60-70% alteration, 30-40% structure
    # Using 65% alteration + 35% structure (common in literature)
    final_mineralization = (
        alteration_norm.multiply(0.65)  # 65% alteration (spectral signatures)
        .add(structural_score.multiply(0.35))  # 35% structural controls
    ).rename('Mineralisation_Probability')
    
    # Calculate percentile-based thresholds for classification
    stats = final_mineralization.reduceRegion(
        reducer=ee.Reducer.percentile([70, 80, 90, 95]),
        geometry=study_area,
        scale=30,
        maxPixels=1e9
    )
    
    p70 = ee.Number(stats.get('p70'))
    p80 = ee.Number(stats.get('p80'))
    p90 = ee.Number(stats.get('p90'))
    p95 = ee.Number(stats.get('p95'))
    
    # Classify into confidence levels
    very_high = final_mineralization.gte(p95).rename('Very_High_Confidence_Mineralisation')
    high = final_mineralization.gte(p90).And(final_mineralization.lt(p95)).rename('High_Confidence_Mineralisation')
    medium = final_mineralization.gte(p80).And(final_mineralization.lt(p90)).rename('Medium_Confidence_Mineralisation')
    low = final_mineralization.gte(p70).And(final_mineralization.lt(p80)).rename('Low_Confidence_Mineralisation')
    all_zones = final_mineralization.gte(p70).rename('All_Mineralisation_Zones')
    
    return ee.Image.cat([
        final_mineralization,
        structural_score,
        very_high,
        high,
        medium,
        low,
        all_zones
    ])

# Check if we have required inputs
if 'alteration_potential_2023_2024' in globals() or 'mineralisation_2023_2024' in globals():
    # Get alteration potential
    if 'alteration_potential_2023_2024' in globals():
        alteration_pot = alteration_potential_2023_2024.select('Alteration_Potential')
    else:
        alteration_pot = mineralisation_2023_2024.select('Mineralisation_Probability')
    
    print("✓ Alteration potential found")
    
    if ('structural_features_classified' in globals() and 
        structural_features_classified is not None and
        'lineament_density_2023_2024' in globals() and
        lineament_density_2023_2024 is not None):
        
        print("✓ Structural features found")
        print("✓ Lineament density found")
        
        # Create final mineralization map
        final_mineralization_2023_2024 = create_final_mineralization_map(
            alteration_pot,
            structural_features_classified,
            lineament_density_2023_2024,
            study_area_buffered
        )
        
        print("\n✓ Final mineralization map created!")
        print("  This incorporates both alteration AND structural controls")
        print("  Following standard practice in mineral exploration literature")
        
        # Calculate visualization parameters (data-driven ranges)
        def calculate_viz_params(image, band_name, study_area, percentile_low=2, percentile_high=98):
            """
            Calculate optimal min/max values for visualization using percentiles
            Excludes outliers for better contrast
            """
            band_image = image.select(band_name)
            stats = band_image.reduceRegion(
                reducer=ee.Reducer.percentile([percentile_low, percentile_high]),
                geometry=study_area,
                scale=30,
                maxPixels=1e9
            )
            # Earth Engine returns keys as '{band_name}_p{percentile}'
            return {
                'min': stats.get(f'{band_name}_p{percentile_low}'),
                'max': stats.get(f'{band_name}_p{percentile_high}')
            }
        
        print("Calculating visualization parameters for final mineralization map...")
        mineralization_viz = calculate_viz_params(
            final_mineralization_2023_2024,
            'Mineralisation_Probability',
            study_area_buffered,
            percentile_low=5,
            percentile_high=95
        )
        structural_viz = calculate_viz_params(
            final_mineralization_2023_2024,
            'Structural_Control_Score',
            study_area_buffered,
            percentile_low=5,
            percentile_high=95
        )
        
        # Visualize final mineralization map with data-driven ranges
        Map.addLayer(final_mineralization_2023_2024.select('Mineralisation_Probability'), {
            'min': mineralization_viz['min'],
            'max': mineralization_viz['max'],
            'palette': ['blue', 'cyan', 'yellow', 'orange', 'red']
        }, 'Final Mineralization Map (Alteration + Structure) - Data-driven range')
        
        Map.addLayer(final_mineralization_2023_2024.select('Very_High_Confidence_Mineralisation'), {
            'min': 0,
            'max': 1,
            'palette': ['transparent', 'darkred']
        }, 'Very High Confidence (Top 5%)', False)
        
        Map.addLayer(final_mineralization_2023_2024.select('High_Confidence_Mineralisation'), {
            'min': 0,
            'max': 1,
            'palette': ['transparent', 'red']
        }, 'High Confidence (Top 10%)', False)
        
        Map.addLayer(final_mineralization_2023_2024.select('Medium_Confidence_Mineralisation'), {
            'min': 0,
            'max': 1,
            'palette': ['transparent', 'orange']
        }, 'Medium Confidence (Top 20%)', False)
        
        Map.addLayer(final_mineralization_2023_2024.select('Structural_Control_Score'), {
            'min': structural_viz['min'],
            'max': structural_viz['max'],
            'palette': ['blue', 'cyan', 'yellow', 'orange', 'red']
        }, 'Structural Control Score - Data-driven range', False)
        
        print("\n" + "=" * 60)
        print("FINAL MINERALIZATION MAP DELIVERABLES")
        print("=" * 60)
        print("✓ Final Mineralization Probability (Alteration + Structure)")
        print("✓ Very High Confidence Zones (Top 5%)")
        print("✓ High Confidence Zones (Top 10%)")
        print("✓ Medium Confidence Zones (Top 20%)")
        print("✓ Structural Control Score")
        print("\nThis represents the FINAL mineralization map following standard practice")
        print("=" * 60)
        
    else:
        print("⚠️  Structural features not available")
        print("   Run mining-palsar.ipynb first to generate structural features")
        final_mineralization_2023_2024 = None
else:
    print("⚠️  Alteration potential not available")
    print("   Run mining-aster.ipynb first to generate alteration potential")
    final_mineralization_2023_2024 = None

Map


In [ ]:
# Export Final Mineralization Maps
# Export all integrated analysis deliverables

def export_final_mineralization_maps():
    """Export final mineralization maps (alteration + structural controls)"""
    
    print("=" * 60)
    print("EXPORTING FINAL MINERALIZATION MAPS")
    print("=" * 60)
    print("Exporting to Google Drive...")
    print("=" * 60)
    
    if 'final_mineralization_2023_2024' in globals() and final_mineralization_2023_2024 is not None:
        geemap.ee_export_image(
            final_mineralization_2023_2024,
            filename='final_mineralization_map_2023_2024.tif',
            scale=30,
            region=study_area_buffered,
            file_per_band=False
        )
        print("✓ final_mineralization_map_2023_2024.tif - Final mineralization map")
        print("  Bands: Mineralisation_Probability, Structural_Control_Score,")
        print("         Very_High_Confidence_Mineralisation, High_Confidence_Mineralisation,")
        print("         Medium_Confidence_Mineralisation, Low_Confidence_Mineralisation,")
        print("         All_Mineralisation_Zones")
    else:
        print("⚠️  No final mineralization map to export. Run Cell 11 first.")
    
    print("\n" + "=" * 60)
    print("EXPORT SUMMARY")
    print("=" * 60)
    print("Final mineralization maps exported to Google Drive")
    print("These represent the complete analysis (alteration + structural controls)")
    print("Following standard practice in mineral exploration literature")
    print("=" * 60)

print("Export function defined. Uncomment and call to export maps.")
print("Example: export_final_mineralization_maps()")
